In [1]:
import json
import requests
from bs4 import BeautifulSoup, Comment
import csv
import pandas as pd
import io
import time
import random

file = open('web_list.txt', 'r') 
web_list = []
for i in file:
    i = i.replace('\n', '')
    web_list.append(i)

def csv_save(CSV_URL):
    global user_list
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"
    headers = {'User-Agent' : random.choice(user_list)}
    s = requests.get(url = CSV_URL,headers = headers,cookies = cookies_dict).text
    data = pd.read_csv(io.StringIO(s))
    url_list = data['URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)']
    detail_list = get_details(url_list)
    data["FULL ADDRESS"] = data['ADDRESS'] + ' ,' + data['CITY'] + ' ,' + data['STATE'] + ' ,' + data['ZIP'].map(str)
    keep_col = ['FULL ADDRESS','PRICE','BEDS','BATHS', 'SQUARE FEET', '$/SQUARE FEET', 'PROPERTY TYPE', 'CITY', 'YEAR BUILT']
    new_f = data[keep_col]
    new_f['detail'] = detail_list
    return new_f
def get_details(url_list):
    global user_list
    headers = {'User-Agent' : random.choice(user_list)}
    detail_list = []
    for url in url_list:
        r = requests.get(url = url,headers = headers,cookies = cookies_dict)
        s = BeautifulSoup(r.text)
        nameList = s.findAll("div" ,{"class":"super-group-content"})
        detail = ''
        for z in nameList:
            titles = z.findAll(class_='title')
            content = z.findAll('li')
            for i,j in zip(titles,content):
                detail += (i.get_text()+j.get_text() + '\n')
        detail_list.append(detail)
        time.sleep(random.uniform(5,10))
    return detail_list
    

In [3]:
import re
def replacenth(string, sub, wanted, n):
    where = [m.start() for m in re.finditer(sub, string)][n-1]
    before = string[:where]
    after = string[where:]
    after = after.replace(sub, wanted, 1)
    newString = before + after
    return newString

def get_menu_page(url):
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36"
    headers = {'User-Agent' : user_agent}
    r = requests.get(url = url, headers = headers).text
    result = set()
    s = BeautifulSoup(r,"lxml")
    possible_list = s.findAll('a')
    menus = ['Menu','menu','MENU','Food & Drink','Locations & Menus']
    end = 0
    for i in possible_list:
        for z in menus:
            if z in i.get_text():
                if i.find_next_sibling() == None:
                    if 'http' not in i.get('href'):
                        result.add(url + i.get('href'))
                        end = 1
                        break    
                    else:
                        result.add(i.get('href'))
                        end =1
                        break
                elif i.find_next_sibling().name == 'a':
                    if 'http' not in i.get('href'):
                        result.add(url + i.get('href'))
                        break
                    else:
                        result.add(i.get('href'))
                        break
                else:
                    next_tag = i.find_next_sibling()
                    while next_tag != None:
                        menu_list = next_tag.findAll('a')
                        for j in menu_list:
                            if 'http' not in j.get('href'):
                                result.add(url + j.get('href'))
                            else:
                                result.add(j.get('href'))
                        next_tag = next_tag.find_next_sibling()
        if end == 1:
            break
    return result

In [4]:
menus_pages = []
for i in web_list[1:]:
    print(i)
    try:
        menus_pages.append(get_menu_page(i))
    except:
        continue

https://locations.wendys.com/boston-ma-10787?utm_source=google&utm_medium=local&utm_campaign=google-distrib
http://machuchickenboston.com/
http://www.nickspizzanseafood.com/
https://www.dunkindonuts.com/?utm_source=google&utm_medium=local&utm_campaign=localmaps&utm_content=308236
http://www.trophyroomboston.com/
https://www.dunkindonuts.com/?utm_source=google&utm_medium=local&utm_campaign=localmaps&utm_content=302673
https://www.dunkindonuts.com/?utm_source=google&utm_medium=local&utm_campaign=localmaps&utm_content=300417
http://www.mymisono.com/
http://www.teresasmarket.com/
http://www.wichitsandwich.com/
http://www.pagliucasrestaurant.com/
http://order.subway.com/Stores/Redirect.aspx?s=54050&sa=0&f=r&utm_source=google&utm_medium=local&utm_term=0&utm_content=54050&utm_campaign=fwh-local-remote-order&cid=0:0:0:0:0:0&segment_code=0
http://www.petespizzaandseafood.com/
http://www.mumbaispiceboston.com/
http://bukkyungrestaurants.com/
http://gensushima.com/
http://spukiesnpizzadorchester.

In [8]:

'''url = 'http://www.backbaysocialclub.com/'
get_menu_page(url)'''

print(menus_pages)

[{'https://www.wendys.com/en-us/kids-meals', 'https://www.wendys.com/en-us/hamburgers', 'https://www.wendys.com/en-us/right-price-right-size', 'https://www.wendys.com/en-us/salads', 'https://www.wendys.com/en-us/breakfast', 'https://www.wendys.com/en-us/beverages', 'https://www.wendys.com/en-us/fries-sides', 'https://www.wendys.com/en-us/chicken', 'https://www.wendys.com/en-us/frosty'}, {'http://machuchickenboston.com//sandwiches', 'http://machuchickenboston.com//drinks', 'http://machuchickenboston.com//side-orders', 'http://machuchickenboston.com//appetizers', 'http://machuchickenboston.com//charcoal-grill/', 'http://machuchickenboston.com//pollos-a-la-brasa', 'http://machuchickenboston.com//salads'}, set(), {'https://www.dunkindonuts.com/?utm_source=google&utm_medium=local&utm_campaign=localmaps&utm_content=308236/en/menu/offers-promotions', 'https://www.dunkindonuts.com/?utm_source=google&utm_medium=local&utm_campaign=localmaps&utm_content=308236#', 'https://www.dunkindonuts.com/?ut

In [9]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(menus_pages, fp)